In [2]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

데이터로드

In [9]:
mnist = tf.keras.datasets.mnist

(x_train , y_train), (x_test, y_test) = mnist.load_data()

정규화

In [10]:
x_train, x_test = x_train /255.0, x_test /255.0

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

데이터 섞고, 묶기

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices(
    (x_test, y_test)).batch(32)

모델설정

In [12]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

model = MyModel()

옵티마이저 설정

In [13]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

손실도와 정확도 함수 설정

In [14]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

1회 학습

In [15]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

1회 테스트 

In [16]:
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

학습 시작

In [17]:
EPOCHS = 5

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = '에포크: {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

에포크: 1, 손실: 0.1397838443517685, 정확도: 95.78833770751953, 테스트 손실: 0.06470540910959244, 테스트 정확도: 97.81999969482422
에포크: 2, 손실: 0.09192144870758057, 정확도: 97.2074966430664, 테스트 손실: 0.05927877500653267, 테스트 정확도: 98.0050048828125
에포크: 3, 손실: 0.06950191408395767, 정확도: 97.87444305419922, 테스트 손실: 0.05572286248207092, 테스트 정확도: 98.13667297363281
에포크: 4, 손실: 0.05595634877681732, 정확도: 98.27249908447266, 테스트 손실: 0.05856676027178764, 테스트 정확도: 98.11499786376953
에포크: 5, 손실: 0.04678685963153839, 정확도: 98.55166625976562, 테스트 손실: 0.060155484825372696, 테스트 정확도: 98.1760025024414
